# UC San Diego: Data Science in Practice - Data Checkpoint
### Summer Session I 2023 | Instructor : C. Alex Simpkins Ph.D.

## Draft project title if you have one (can be changed later)

# Names

- Harshita Saha
- Conner Hsu
- Anastasiya Markova
- Sidharth Srinath

<a id='research_question'></a>
# Research Question

* One sentence that describes the question you address in your project. Make sure what you’re measuring (variables) to answer your question is clear!

---

Given clinical data for cohorts with and without diabetes, which feature most correlates to the measure of 'blood_glucose_level' for each cohort, and are they different between the cohorts?

# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

---

- Dataset Name: Diabetes prediction dataset
- Link to the dataset: https://www.kaggle.com/datasets/iammustafatz/diabetes-prediction-dataset
- Number of observations: 100,000

Description: 

This dataset is a collection of medical and demographic data sourced from Electronic Health Records (EHRs), collected by medical professionals and institutions. The Diabetes Prediction Dataset is built off of multiple EHRs that were collected and combined together

This dataset contains records for 100,000 patients, and contains data regarding whether the patient has `diabetes`, `hypertension`, and `heart_disease`, as well as their `age`, `gender`, `smkoing_history`, `bmi`, `HbA1c_level`, and `blood_glucose_level`.


# Data Wrangling

* Explain steps taken to pull the data you need into Python.

---

Since our dataset is sourced from Kaggle, we will use the steps below to pull the data we need into Python:

1. Manually download the dataset from [this link](https://www.kaggle.com/datasets/iammustafatz/diabetes-prediction-dataset?resource=download) and save it as `diabetes_prediction_dataset.csv`, in the same directory as the working notebook. 

2. Use the pandas `read_csv()` function to read in the data in to a Dataframe, as displayed below. 

In [1]:
# import relevant packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as pyplot
import seaborn as sns


In [2]:
# reading in the data to a DataFrame
df = pd.read_csv('diabetes_prediction_dataset.csv')
df.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


# Data Cleaning

Describe your data cleaning steps here.

In [3]:
# check if there is any missing data
df.isna().sum().sum()

0

In [4]:
# check the datatypes of each column
df.dtypes

gender                  object
age                    float64
hypertension             int64
heart_disease            int64
smoking_history         object
bmi                    float64
HbA1c_level            float64
blood_glucose_level      int64
diabetes                 int64
dtype: object

In [5]:
# checking gender of patients
df.groupby('gender').count()['age']

gender
Female    58552
Male      41430
Other        18
Name: age, dtype: int64

In [6]:
# removing records where gender is not specified/recorded as it may affect predictions
df = df[df['gender'] != 'Other']

In [7]:
# removing records for patients less than 20
# this is because the rates of diabetes are lower in these populations
# and diabetes is less well studied as a result, and analyis based 
# on these groups might not generalizable
# https://www.cdc.gov/diabetes/data/statistics-report/newly-diagnosed-diabetes.html
# https://diabetesjournals.org/care/article/46/3/490/148482/Youth-Onset-Type-2-Diabetes-The-Epidemiology-of-an
df = df[df['age'] >= 20]
df.shape

(80321, 9)

In [8]:
# add in columns to label the levels for `bmi`, `HbA1c_level`, and `blood_glucose_level`
# bmi cutoffs: https://www.cdc.gov/obesity/basics/adult-defining.html
# HbA1c_level cutoffs: https://bmcendocrdisord.biomedcentral.com/articles/10.1186/s12902-019-0338-7
# blood_glucose_level cutoffs: https://medlineplus.gov/ency/patientinstructions/000086.htm


In [9]:
# bmi labeling
bmi_bins = pd.cut(df['bmi'], [0, 18.5, 25, 30, round(df['bmi'].max())], right=False, \
                  labels=['underweight', 'healthy', 'overweight', 'obese']) 
df['bmi_label'] = bmi_bins

In [10]:
# blood_glucose labeling
bg_bins = pd.cut(df['blood_glucose_level'], [0, 90, 130, round(df['blood_glucose_level'].max())], right=False, \
                 labels=['low', 'normal', 'high'])
df['bg_label'] = bg_bins

In [11]:
# HbA1c labeling
age_bins = pd.cut(df['age'], [19, 39, 59, round(df['age'].max())], labels=['g1', 'g2', 'g3'])
df['age_label'] = age_bins

In [26]:
#setting up thresholds for HbA1c levels
conditions = [df['age_label']=='g1', df['age_label']=='g2', df['age_label']=='g3']
choices = [6.0, 6.1, 6.5]
HbA1c_exp = np.select(conditions, choices)
df['HbA1c_label'] = HbA1c_exp

In [27]:
#assigning high/normal based on the thresholds and current HbA1c levels
df['HbA1c_label'] = (df['HbA1c_level'] <= df['HbA1c_label']).replace({True:'normal', False:'high'})
df.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes,bmi_label,bg_label,age_label,HbA1c_label
0,Female,80.0,0,1,never,25.19,6.6,140,0,overweight,high,g3,high
1,Female,54.0,0,0,No Info,27.32,6.6,80,0,overweight,low,g2,high
2,Male,28.0,0,0,never,27.32,5.7,158,0,overweight,high,g1,normal
3,Female,36.0,0,0,current,23.45,5.0,155,0,healthy,high,g1,normal
4,Male,76.0,1,1,current,20.14,4.8,155,0,healthy,high,g3,normal


In [28]:
#renaming the columns
df = df[list(df.columns[:6])+['bmi_label', 'HbA1c_level', 'HbA1c_label', 'blood_glucose_level', 'bg_label', 'diabetes']]
#reordering the columns
df.columns=[list(df.columns[:7])+['hba1c', 'hba1c_label', 'bg', 'bg_label', 'diabetes']]
df.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,bmi_label,hba1c,hba1c_label,bg,bg_label,diabetes
0,Female,80.0,0,1,never,25.19,overweight,6.6,high,140,high,0
1,Female,54.0,0,0,No Info,27.32,overweight,6.6,high,80,low,0
2,Male,28.0,0,0,never,27.32,overweight,5.7,normal,158,high,0
3,Female,36.0,0,0,current,23.45,healthy,5.0,normal,155,high,0
4,Male,76.0,1,1,current,20.14,healthy,4.8,normal,155,high,0
